# 신경망의 톱니바퀴: 텐서 연산

### 원소별 연산

케라스의 keras.layers.Dense(512, activation='relu') 이 층은 행렬을 입력으로 받고 입력 텐서의 새로운 표현인 또 다른 행렬을 반환하는 함수처럼 해석할 수 있다.  
구체적으로 보면 이 함수는 다음과 같다.  
output = relu(dot(W, input) + b)  
여기에는 세가지의 텐서 연산이 있다.  
1. 입력 텐서와  W 사이의 dot production  
2. dot production 으로 만들어진 행렬과 벡터 b 사이의 덧셈  
3. relu(렐루)연산. relu(x)는 max(x, 0)이다.

In [1]:
def naive_relu(x):
    assert len(x.shape) == 2 # x는 랭크 2의 넘파이 배열이다.
    x = x.copy() # 입력 텐서 자체를 바꾸지 않도록 복사한다.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

In [2]:
# 덧셈도 동일한 방법으로 합니다.
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

넘파이는 위와 같은 원소별 연산을 빠르게 처리합니다.

In [3]:
import time
import numpy as np

x = np.random.random((20, 100))
y = np.random.random((20, 100))

t0 = time.time()
for _ in range(1000):
    z = x + y
    z = np.maximum(z, 0.)
print("걸린 시간: {0:.2f} s".format(time.time() - t0))

걸린 시간: 0.00 s


In [4]:
t0 = time.time()
for _ in range(1000):
    z = naive_add(x, y)
    z = naive_relu(z)
print("걸린 시간: {0:.2f} s".format(time.time() - t0))

걸린 시간: 1.17 s


넘파이 연산을 한 것과 하지 않은 방법의 걸리는 시간 차이가 꽤 큽니다.

# 브로드캐스팅

작은 텐서가 큰 텐서의 크기에 맞추어 "브로드캐스팅"을 하는데, 브로드캐스팅은 크게 두 단계로 나뉩니다.  
1. 큰 텐서의 ndim에 맞도록 작은 텐서에 (브로드캐스팅 축이라고 부르는) 축이 추가됩니다.  
2. 작은 텐서가 새 축을 따라서 큰 텐서의 크기에 맞도록 반복됩니다.

In [5]:
import numpy as np
X = np.random.random((32, 10)) # x는 크기가 (32,10)인 랜덤한 행렬입니다.
y = np.random.random((10,)) # y는  크기가 (10,) 인 랜덤한 행렬입니다.

In [6]:
y = np.expand_dims(y, axis=0) # y에 비어있는 첫 번째 축을 추가하여 크기를 (1,10)으로 만듭니다.

In [7]:
Y = np.concatenate([y] * 32, axis=0) 

np.concatenate() 메서드는 배열을 합치는데 사용됩니다. 기존 배열들을 지정한 축(axis)을 따라 연결하거나 결합합니다.  
axis=0이면 열 방향으로 배열을 연결합니다.  
즉, y를 32번 반복하여 y의 크기가 (32,10)으로 됩니다.

단순하게 구현한 예를 살펴봅니다.

In [8]:
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2 # x는 랭크-2 넘파이 배열입니다.
    assert len(y.shape) == 1 # y는 넘파이 벡터 입니다.
    assert x.shape[1] == y.shape[0] # x의 두 번째 차원의 크기와 y의 첫 번째 차원의 크기가 같지 않으면 에러를 발생 시킵니다.
    x = x.copy() # 입력 텐서 자체를 바꾸지 않도록 복사합니다.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[j] # 행렬과 벡터를 더하는 실질적인 연산입니다.
    return x

즉, 위 함수는 행렬 x와 벡터 y를 더하여 결과를 반환하는 함수입니다.

In [9]:
import numpy as np
x = np.random.random((64, 3, 32, 10))
y = np.random.random((32, 10))
z = np.maximum(x, y)
print(z.shape)

(64, 3, 32, 10)


z의 shape는 (64, 3, 32, 10)입니다.

# 텐서 곱셈

텐서 곱셈은 다른말로 점곱(dot production)이라고 합니다.

In [10]:
def naive_vector_dot(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]
    z = 0.
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z

두 벡터의 점곱은 스칼라가 되므로 원소 개수가 같은 벡터끼리 점곱이 가능합니다.  
행렬 x와 벡터 y 사이에서도 점곱이 가능합니다. y와 x의 행 사이에서 점곱이 일어나므로 벡터가 반환됩니다. 다음과 같이 구현할 수 있습니다.  

In [11]:
def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i] += x[i, j] * y[j] # 여기가 실질적인 dot production이 이루어지는 곳입니다.
    return z

In [12]:
def naive_matrix_vector_dot(x, y):
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i, :], y)
    return z

두 텐서중 하나라도 ndim이 1보다 크면 dot 연산에 교환법칙이 성립되지 않습니다.  
dot(x,y) 와 dot(y,x)이 다르다는 이야기입니다.  
x.shape[1] == y.shape[0]일 때, 두 행렬 x와 y의 점곱 (dot(x, y))이 성립됩니다. x의 행과 y의 열 사이 벡터 점곱으로 인해 (x.shape[0], y.shape[1]) 크기의 행렬이 됩니다.

In [13]:
def naive_matrix_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0] #  x의 두 번째 차원이 y의 첫 번째 차원과 같아야 합니다.
    z = np.zeros((x.shape[0], y.shape[1])) # 이 연산은 0이 채워진 특정 크기의 벡터를 만듭니다.
    for i in range(x.shape[0]): # x의 행을 반복합니다.
        for j in range(y.shape[1]): # y의 열을 반복합니다.
            row_x = x[i, :]
            column_y = y[:, j]
            z[i, j] = naive_vector_dot(row_x, column_y)
    return z

In [14]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 텐서 크기 변환

In [15]:
train_images = train_images.reshape((60000, 28 * 28))

In [16]:
x = np.array([[0., 1.],
             [2., 3.],
             [4., 5.]])
x.shape

(3, 2)

In [17]:
x = x.reshape((6, 1))
x

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.]])

텐서의 크기를 변환 한다는 것은 특정 크기에 맞게 열과 행을 재배치 한다는 것입니다. 
위 예제를 보면, (3,2)의 크기에서 (6,1)로 재배열을 했지만, (4,3)이나, (6,2)의 재배열은 불가능 합니다.  
(2,3) 또는 (3,2) 크기로 재배열은 가능합니다.

In [18]:
x = x.reshape((2, 3))
x

array([[0., 1., 2.],
       [3., 4., 5.]])

In [19]:
x = np.zeros((300, 20))
x = np.transpose(x)
x.shape

(20, 300)

가장 많이 사용하는 재배열은 전치(Transpose)입니다. 

# 신경망의 엔진: 그레디언트 기반 최적화

### 텐서플로의 그레디언트 테이프

In [32]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
x = tf.Variable(0.) # 초기값 0으로 스칼라 변수를 생성합니다.
with tf.GradientTape() as tape: # GradientTape 블록을 생성합니다.
    y = 2 * x + 3 # 이 블록 안에서 텐서 연산을 적용합니다.
grad_of_y_wrt_x = tape.gradient(y, x) # tape를 사용해서 변수 x에 대한 출력 y의 그레디언트를 생성합니다.

텐서플로우에서 제공하는 자동 미분 기능을 활용할 수 있는 API는 GradientTape() 입니다.  
이 API는 파이썬의 with 문과 함께 사용하여 해당 코드 블록 안의 모든 텐서 연산을 계산 그래프 형태로 기록합니다.  
그 다음 이 그래프를 이용해서 (tf.Variable 클래스의 인스턴스인) 변수 또는 변수 집합에 대한 어떤 출력의 그레디언트도 계산할 수 있습니다.  
tf.Variable은 변경 가능한(mutable) 상태를 담기 위한 특별한 종류의 텐서입니다. 예를 들어 신경망의 가중치는 항상 tf.Variable의 인스턴스입니다.  

In [22]:
x = tf.Variable(tf.zeros((2, 2)))
with tf.GradientTape() as tape:
    y = 2 * x + 3
grad_of_y_wrt_x = tape.gradient(y, x)

GradientTape()를 다차원 텐서와 함께 사용 가능합니다.  

In [24]:
W = tf.Variable(tf.random.uniform((2, 2)))
b = tf.Variable(tf.zeros((2,)))
x = tf.random.uniform((2, 2))
with tf.GradientTape() as tape:
    y = tf.matmul(x, W) + b # 행렬 x와 가중치 W를 곱하고 편향 b를 더하는 식입니다. matnul()메서드는 행렬 곱셈을 연산합니다.  
grad_of_y_wrt_W_and_b = tape.gradient(y, [W, b])

변수 리스트의 그레디언트를 계산할 수도 있습니다.

# 첫 번째 예제 다시 살펴보기

In [26]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28)) # train_images의 shape을 (60000, 784)로 재배열
train_images = train_images.astype("float32") / 255 # 데이터 타입을 "float32"로 변환
test_images = test_images.reshape((10000, 28 * 28)) # test_images의 shape을 (10000, 784)로 재배열
test_images = test_images.astype("float32") / 255 # 데이터 타입을 "float32"로 변환 후, 255로 나누어 출력을 0~1사이의 값으로 만듦

In [33]:
model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])

위 모델은 두 개의 Dense층이 연결되어있고 각 층은 가중치 텐서를 포함하여 입력 데이터에 대한 몇 개의 간단한 텐서 연산을 적용합니다.

In [34]:
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

모델을 컴파일 합니다. 옵티마이저는 "rmsprop"을 사용하여 그레디언트를 계산하고, loss function은 "sparse_categorical_crossentropy" 를 사용합니다. 

In [35]:
model.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 1s 2ms/step - loss: 0.2578 - accuracy: 0.9244
Epoch 2/5
469/469 [==============================] - 1s 1ms/step - loss: 0.1019 - accuracy: 0.9695
Epoch 3/5
469/469 [==============================] - 1s 1ms/step - loss: 0.0680 - accuracy: 0.9803
Epoch 4/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0491 - accuracy: 0.9849
Epoch 5/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0370 - accuracy: 0.9890


fit 메소드를 호출하면, 모델이 128개 샘플의 미니배치로 훈련 데이터를 5번 반복합니다.  
각 배치에서 모델이 가중치에 대한 손실의 그레이디언트를 계산합니다. 그다음 이 배치에서 손실 값을 감소시키는 방향으로 가중치를 이동시킵니다.  
다섯 번의 에포크 동안 모델은 2,345번의 그레이디언트 업데이트를 수행할 것입니다(에포크마다 469번)

## 텐서플로를 사용해 첫 번째 예제를 밑바닥부터 다시 구현하기

### 단순한 Dense 클래스

In [36]:
import tensorflow as tf

class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation

        w_shape = (input_size, output_size) # 랜덤한 값으로 초기화 된 (imput_size, output_size)크기의 행렬 W를 만듭니다.
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)

        b_shape = (output_size,) # 0으로 초기화 된 (output_size,)크기의 벡터 b를 만듭니다.
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs): # 정방행으로 forwardpass를 수행합니다.
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property 
    def weights(self): # 층의 가중치를 추출하기 위한 메서드
        return [self.W, self.b]

### 단순한 Sequential 클래스

층의 리스트를 받고 __call__()메서드에서 입력을 사용하여 층을 순서대로 호출합니다.  
층의 파라미터를 쉽게 구할 수 있도록 weights속성을 제공합니다.

In [37]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
           x = layer(x)
        return x

    @property
    def weights(self):
       weights = []
       for layer in self.layers:
           weights += layer.weights
       return weights

위 두 클래스를 가지고 kears와 유사한 모델을 만들 수 있습니다.

In [38]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu), # 첫 번째 층 input: 784, output: 512, 활성화 함수: relu
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax) # 두 번째 층 input: 512, output: 10, 활성화 함수: softmax
])
assert len(model.weights) == 4

### 배치 제너레이터

In [39]:
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128): # 초기화 함수
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

위 클래스는 미니 배치를 만드는 클래스 입니다.  

### 훈련 스텝 실행하기

In [41]:
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape: # 정방향 패스를 실행합니다. (GradientTape 블록 안에서 모델의 예측을 계산합니다)
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights) # 가중치에 대한 손실의 그레이디언트를 계산합니다. gradients 리스트의 각 항목은 model.weights 리스트에 있는 가중치에 매칭됩니다.
    update_weights(gradients, model.weights) # 이 그레디언트를 사용하여 가중치를 업데이트 합니다.
    return average_loss

한 배치 데이터에서 모델을 실행하고 가중치를 업데이트 하는 일입니다.  이를 실행하기 위해선 다음이 필요합니다.  
1. 배치에 있는 이미지에 대해 모델의 예측을 계산합니다.  
2. 실제 레이블을 사용하여 이 예측의 손실 값을 계산합니다.  
3. 모델 가중치에 대한 손실의 그레이디언트를 계산합니다.  
4. 이 그레이디언트의 반대 방향으로 가중치를 조금 이동합니다.  
GradientTape() 메서드를 사용하여 그레디언트를 계산 합니다.

In [43]:
learning_rate = 1e-3

def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate) # 텐서플로 변수의 assign_sub 메서드는 -=와 동일합니다.

가중치 업데이트의 목적은 배치의 손실을 감소시키기 위한 방향으로 가중치를 약간 이동시키는 것 입니다.  
이동의 크기는 학습률(learning_rate)에 의해 결정됩니다.  
update_weights 함수를 구현하는 가장 간단한 방법은 각 가중치에서 gradient * learning_rate를 빼는 것입니다.

In [44]:
from tensorflow.keras import optimizers

optimizer = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

가중치 업데이트는 수동으로 구현하는 경우는 거의 없습니다. 대신에 케라스의 optimizers 인스턴스를 사용합니다.

In [45]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"에포크 {epoch_counter}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"{batch_counter}번째 배치 손실: {loss:.2f}")

배치 카운터에서 미니 배치를 학습 시키고, 배치 카운터가 100%가 되면 for 문을 빠져 나갑니다.  
이런식으로 배치 카운터가 한 번 반복되면, epoch가 1 카운트 됩니다.

In [46]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

에포크 0
0번째 배치 손실: 4.67
100번째 배치 손실: 2.23
200번째 배치 손실: 2.21
300번째 배치 손실: 2.10
400번째 배치 손실: 2.19
에포크 1
0번째 배치 손실: 1.92
100번째 배치 손실: 1.87
200번째 배치 손실: 1.82
300번째 배치 손실: 1.71
400번째 배치 손실: 1.80
에포크 2
0번째 배치 손실: 1.59
100번째 배치 손실: 1.56
200번째 배치 손실: 1.50
300번째 배치 손실: 1.43
400번째 배치 손실: 1.49
에포크 3
0번째 배치 손실: 1.33
100번째 배치 손실: 1.32
200번째 배치 손실: 1.24
300번째 배치 손실: 1.21
400번째 배치 손실: 1.26
에포크 4
0번째 배치 손실: 1.14
100번째 배치 손실: 1.14
200번째 배치 손실: 1.04
300번째 배치 손실: 1.05
400번째 배치 손실: 1.10
에포크 5
0번째 배치 손실: 0.99
100번째 배치 손실: 1.00
200번째 배치 손실: 0.90
300번째 배치 손실: 0.93
400번째 배치 손실: 0.98
에포크 6
0번째 배치 손실: 0.88
100번째 배치 손실: 0.90
200번째 배치 손실: 0.80
300번째 배치 손실: 0.84
400번째 배치 손실: 0.89
에포크 7
0번째 배치 손실: 0.80
100번째 배치 손실: 0.81
200번째 배치 손실: 0.72
300번째 배치 손실: 0.77
400번째 배치 손실: 0.83
에포크 8
0번째 배치 손실: 0.73
100번째 배치 손실: 0.74
200번째 배치 손실: 0.66
300번째 배치 손실: 0.71
400번째 배치 손실: 0.78
에포크 9
0번째 배치 손실: 0.68
100번째 배치 손실: 0.69
200번째 배치 손실: 0.61
300번째 배치 손실: 0.67
400번째 배치 손실: 0.73


mnist로부터 데이터를 다운 받아서, train_data와 test_data를 나눈후, 각 데이터들을 재배열 하여 fit 함수를 통해 학습 합니다.

# 모델 평가하기

In [47]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1) # argmax함수를 사용하여 각 에측된 확률 분포에서 가장 높은 확률을 가진 클래스의 인덱스를 선택합니다.
matches = predicted_labels == test_labels # 측된 레이블과 실제 레이블을 비교하여 일치 여부를 나타내는 불리언 배열을 생성합니다. predicted_labels는 모델이 예측한 레이블이고, test_labels는 실제 레이블입니다.
print(f"정확도: {matches.mean():.2f}")

정확도: 0.82
